# The event sourcing
- The Event Sourcing pattern stores state changes as a sequence of events, allowing the reconstruction of past states and providing an audit trail. This pattern is particularly useful in systems where the state is complex and the business rules for transitions are complex.
- As we will see in implementation examples later, the Event Sourcing pattern emphasizes the importance of capturing all changes to an application state as a sequence of events. An outcome of this is that the application state can be reconstructed at any point in time by replaying these events.

- There are several real-world examples in the software category:
  - Audit trails: Keeping a record of all changes made to a database for compliance
  - Collaborative editing: Allowing multiple users to edit a document simultaneously
  - Undo/redo features: Providing the ability to undo or redo actions in an application

In [1]:
class Account:
    def __init__(self):
        self.balance = 0
        self.events = []

    def apply_event(self, event):
        if event["type"] == "deposited":
            self.balance += event["amount"]
        elif event["type"] == "withdrawn":
            self.balance -= event["amount"]
        self.events.append(event)

    def deposit(self, amount):
        event = {"type": "deposited", "amount": amount}
        self.apply_event(event)

    def withdraw(self, amount):
        event = {"type": "withdrawn", "amount": amount}
        self.apply_event(event)

In [2]:
def main():
    account = Account()
    account.deposit(100)
    account.deposit(50)
    account.withdraw(30)
    account.deposit(30)

    for evt in account.events:
        print(evt)
    print(f"Balance: {account.balance}")

In [3]:
main()

{'type': 'deposited', 'amount': 100}
{'type': 'deposited', 'amount': 50}
{'type': 'withdrawn', 'amount': 30}
{'type': 'deposited', 'amount': 30}
Balance: 150


<hr>

In [6]:
!pip install eventsourcing

In [7]:
from eventsourcing.domain import Aggregate, event
from eventsourcing.application import Application

In [8]:
class InventoryItem(Aggregate):
    @event("ItemCreated")
    def __init__(self, name, quantity=0):
        self.name = name
        self.quantity = quantity

    @event("QuantityIncreased")
    def increase_quantity(self, amount):
        self.quantity += amount

    @event("QuantityDecreased")
    def decrease_quantity(self, amount):
        self.quantity -= amount

In [9]:
class InventoryApp(Application):
    def create_item(self, name, quantity):
        item = InventoryItem(name, quantity)
        self.save(item)
        return item.id

    def increase_item_quantity(self, item_id, amount):
        item = self.repository.get(item_id)
        item.increase_quantity(amount)
        self.save(item)

    def decrease_item_quantity(self, item_id, amount):
        item = self.repository.get(item_id)
        item.decrease_quantity(amount)
        self.save(item)

In [10]:
def main():
    app = InventoryApp()

    # Create a new item
    item_id = app.create_item("Laptop", 10)
    # Increase quantity
    app.increase_item_quantity(item_id, 5)
    # Decrease quantity
    app.decrease_item_quantity(item_id, 3)

    notifs = app.notification_log.select(start=1, limit=5)
    notifs = [notif.state for notif in notifs]
    for notif in notifs:
        print(notif.decode())

In [11]:
main()

{"timestamp":{"_type_":"datetime_iso","_data_":"2025-01-29T12:41:59.543282+00:00"},"originator_topic":"__main__:InventoryItem","name":"Laptop","quantity":10}
{"timestamp":{"_type_":"datetime_iso","_data_":"2025-01-29T12:41:59.543568+00:00"},"amount":5}
{"timestamp":{"_type_":"datetime_iso","_data_":"2025-01-29T12:41:59.543740+00:00"},"amount":3}
